In [1]:
from plotnine import *
import pandas as pd
import glob

In [26]:
RESULTS_DIRECTORY = '../results/'
DATA_DIRECTORY = f'{RESULTS_DIRECTORY}5_classifiers/'
CSV_FILENAME = 'simple_classifiers.csv'

In [3]:
!python --version

Python 3.6.6 :: Anaconda, Inc.


In [4]:
!pip freeze | grep nine

plotnine==0.4.0


In [5]:
!tree

.
├── 5_classifiers
│   ├── Artifact.csv
│   ├── Background ring.csv
│   ├── Diffuse scattering.csv
│   ├── Ice ring.csv
│   ├── Loop scattering.csv
│   ├── Non-uniform detector.csv
│   └── Strong background.csv
├── ClassificationTry.ipynb
├── LightGBM.ipynb
├── playground
│   ├── create_joint_vector.ipynb
│   └── sandbox.ipynb
├── plotnine.ipynb
├── radial_compression_classifier.ipynb
├── refleX - preprocessing pipeline.ipynb
├── RFC_DTC
│   ├── Artifact.csv
│   ├── Background ring.csv
│   ├── Diffuse scattering.csv
│   ├── Ice ring.csv
│   ├── Loop scattering.csv
│   ├── Non-uniform detector.csv
│   └── Strong background.csv
├── wykresik.png
└── wykresik.svg

3 directories, 23 files


In [6]:
file_paths = glob.glob(f'{DATA_DIRECTORY}*')

In [9]:
def gather( df, key, value, cols ):
    id_vars = [ col for col in df.columns if col not in cols ]
    id_values = cols
    var_name = key
    value_name = value
    return pd.melt( df, id_vars, id_values, var_name, value_name )

In [14]:
joined_df = pd.DataFrame()
for file_path in file_paths:
    df = pd.read_csv(file_path, index_col=0)
    df['anomaly'] = file_path[file_path.rfind('/')+1:file_path.rfind('.')]
    joined_df = joined_df.append(df, ignore_index=True)
joined_df = gather(joined_df, 'measure_name', 'measure_value', ['accuracy', 'precision', 'recall', 'recall_macro', 'roc_auc', 'f1'])
joined_df

,clf,fit_time,fold,score_time,anomaly,measure_name,measure_value
0,RFC,0.896713,0,0.617316,Non-uniform detector,accuracy,0.839450
1,RFC,0.904699,1,0.618329,Non-uniform detector,accuracy,0.839450
2,RFC,1.005275,2,0.618923,Non-uniform detector,accuracy,0.830275
3,RFC,0.904606,3,0.617886,Non-uniform detector,accuracy,0.848624
4,RFC,0.902541,4,0.618072,Non-uniform detector,accuracy,0.838710
5,RFC,0.897909,5,0.618293,Non-uniform detector,accuracy,0.857143
6,RFC,0.898440,6,0.618423,Non-uniform detector,accuracy,0.847926
7,RFC,0.908759,7,0.616907,Non-uniform detector,accuracy,0.834101
8,RFC,0.899081,8,0.617618,Non-uniform detector,accuracy,0.843318
9,RFC,1.006105,9,0.618804,Non-uniform detector,accuracy,0.857143


In [25]:
mean_df = joined_df.groupby(['measure_name', 'clf', 'anomaly'])['measure_value'].mean().reset_index()
std_df = joined_df.groupby(['measure_name', 'clf', 'anomaly'])['measure_value'].std().reset_index()

aggregated = mean_df.rename(index=str, columns={'measure_value':'mean'})
aggregated.loc[:,'std'] = std_df.loc[:, 'measure_value'].values
aggregated 

,measure_name,clf,anomaly,mean,std
0,accuracy,DTC,Artifact,0.897464,0.020694
1,accuracy,DTC,Background ring,0.671155,0.035809
2,accuracy,DTC,Diffuse scattering,0.796234,0.018470
3,accuracy,DTC,Ice ring,0.876254,0.013743
4,accuracy,DTC,Loop scattering,0.628330,0.023657
5,accuracy,DTC,Non-uniform detector,0.763119,0.035593
6,accuracy,DTC,Strong background,0.862918,0.025384
7,accuracy,GaussianNB,Artifact,0.797209,0.028570
8,accuracy,GaussianNB,Background ring,0.621476,0.034940
9,accuracy,GaussianNB,Diffuse scattering,0.506942,0.036872


In [27]:
aggregated.to_csv(f'{RESULTS_DIRECTORY}{CSV_FILENAME}')

In [29]:
paul_tol_colors = ['#4477AA', '#66CCEE', '#228833', '#CCBB44', '#EE6677', '#AA3377', '#BBBBBB']

p = ggplot(aggregated, aes(x='clf', y='mean', fill='clf')) + geom_point() + scale_fill_manual(paul_tol_colors)
p = p + geom_bar(stat='identity')
p = p + geom_errorbar(aes(ymin='mean-std', ymax='mean+std'))
p = p + facet_grid('anomaly~measure_name') + theme_minimal()
ggsave(p, f'{RESULTS_DIRECTORY}wykresik.svg')
ggsave(p, f'{RESULTS_DIRECTORY}wykresik.png', width=15, height=15)

/home/reflex/.conda/envs/fastai/lib/python3.6/site-packages/plotnine/coords/coord_cartesian.py:31: MatplotlibDeprecationWarning: The Bunch class was deprecated in Matplotlib 3.0 and will be removed in 3.2. Use types.SimpleNamespace instead.
  self.limits = Bunch(xlim=xlim, ylim=ylim)
/home/reflex/.conda/envs/fastai/lib/python3.6/copy.py:274: MatplotlibDeprecationWarning: The Bunch class was deprecated in Matplotlib 3.0 and will be removed in 3.2. Use types.SimpleNamespace instead.
  y = func(*args)
/home/reflex/.conda/envs/fastai/lib/python3.6/copy.py:274: MatplotlibDeprecationWarning: The Bunch class was deprecated in Matplotlib 3.0 and will be removed in 3.2. Use types.SimpleNamespace instead.
  y = func(*args)
/home/reflex/.conda/envs/fastai/lib/python3.6/copy.py:274: MatplotlibDeprecationWarning: The Bunch class was deprecated in Matplotlib 3.0 and will be removed in 3.2. Use types.SimpleNamespace instead.
  y = func(*args)
/home/reflex/.conda/envs/fastai/lib/python3.6/copy.py:274:

/home/reflex/.conda/envs/fastai/lib/python3.6/site-packages/plotnine/facets/layout.py:147: MatplotlibDeprecationWarning: The Bunch class was deprecated in Matplotlib 3.0 and will be removed in 3.2. Use types.SimpleNamespace instead.
  return Bunch(x=xsc, y=ysc)
/home/reflex/.conda/envs/fastai/lib/python3.6/site-packages/plotnine/facets/layout.py:147: MatplotlibDeprecationWarning: The Bunch class was deprecated in Matplotlib 3.0 and will be removed in 3.2. Use types.SimpleNamespace instead.
  return Bunch(x=xsc, y=ysc)
/home/reflex/.conda/envs/fastai/lib/python3.6/site-packages/plotnine/facets/layout.py:147: MatplotlibDeprecationWarning: The Bunch class was deprecated in Matplotlib 3.0 and will be removed in 3.2. Use types.SimpleNamespace instead.
  return Bunch(x=xsc, y=ysc)
/home/reflex/.conda/envs/fastai/lib/python3.6/site-packages/plotnine/facets/layout.py:147: MatplotlibDeprecationWarning: The Bunch class was deprecated in Matplotlib 3.0 and will be removed in 3.2. Use types.Simple

/home/reflex/.conda/envs/fastai/lib/python3.6/site-packages/plotnine/facets/layout.py:147: MatplotlibDeprecationWarning: The Bunch class was deprecated in Matplotlib 3.0 and will be removed in 3.2. Use types.SimpleNamespace instead.
  return Bunch(x=xsc, y=ysc)
/home/reflex/.conda/envs/fastai/lib/python3.6/site-packages/plotnine/facets/layout.py:147: MatplotlibDeprecationWarning: The Bunch class was deprecated in Matplotlib 3.0 and will be removed in 3.2. Use types.SimpleNamespace instead.
  return Bunch(x=xsc, y=ysc)
/home/reflex/.conda/envs/fastai/lib/python3.6/site-packages/plotnine/facets/layout.py:147: MatplotlibDeprecationWarning: The Bunch class was deprecated in Matplotlib 3.0 and will be removed in 3.2. Use types.SimpleNamespace instead.
  return Bunch(x=xsc, y=ysc)
/home/reflex/.conda/envs/fastai/lib/python3.6/site-packages/plotnine/facets/layout.py:147: MatplotlibDeprecationWarning: The Bunch class was deprecated in Matplotlib 3.0 and will be removed in 3.2. Use types.Simple

/home/reflex/.conda/envs/fastai/lib/python3.6/site-packages/plotnine/facets/layout.py:147: MatplotlibDeprecationWarning: The Bunch class was deprecated in Matplotlib 3.0 and will be removed in 3.2. Use types.SimpleNamespace instead.
  return Bunch(x=xsc, y=ysc)
/home/reflex/.conda/envs/fastai/lib/python3.6/site-packages/plotnine/facets/layout.py:147: MatplotlibDeprecationWarning: The Bunch class was deprecated in Matplotlib 3.0 and will be removed in 3.2. Use types.SimpleNamespace instead.
  return Bunch(x=xsc, y=ysc)
/home/reflex/.conda/envs/fastai/lib/python3.6/site-packages/plotnine/facets/layout.py:147: MatplotlibDeprecationWarning: The Bunch class was deprecated in Matplotlib 3.0 and will be removed in 3.2. Use types.SimpleNamespace instead.
  return Bunch(x=xsc, y=ysc)
/home/reflex/.conda/envs/fastai/lib/python3.6/site-packages/plotnine/facets/layout.py:147: MatplotlibDeprecationWarning: The Bunch class was deprecated in Matplotlib 3.0 and will be removed in 3.2. Use types.Simple

/home/reflex/.conda/envs/fastai/lib/python3.6/site-packages/plotnine/facets/layout.py:147: MatplotlibDeprecationWarning: The Bunch class was deprecated in Matplotlib 3.0 and will be removed in 3.2. Use types.SimpleNamespace instead.
  return Bunch(x=xsc, y=ysc)
/home/reflex/.conda/envs/fastai/lib/python3.6/site-packages/plotnine/facets/layout.py:147: MatplotlibDeprecationWarning: The Bunch class was deprecated in Matplotlib 3.0 and will be removed in 3.2. Use types.SimpleNamespace instead.
  return Bunch(x=xsc, y=ysc)
/home/reflex/.conda/envs/fastai/lib/python3.6/site-packages/plotnine/facets/layout.py:147: MatplotlibDeprecationWarning: The Bunch class was deprecated in Matplotlib 3.0 and will be removed in 3.2. Use types.SimpleNamespace instead.
  return Bunch(x=xsc, y=ysc)
/home/reflex/.conda/envs/fastai/lib/python3.6/site-packages/plotnine/facets/layout.py:147: MatplotlibDeprecationWarning: The Bunch class was deprecated in Matplotlib 3.0 and will be removed in 3.2. Use types.Simple

/home/reflex/.conda/envs/fastai/lib/python3.6/site-packages/plotnine/facets/layout.py:147: MatplotlibDeprecationWarning: The Bunch class was deprecated in Matplotlib 3.0 and will be removed in 3.2. Use types.SimpleNamespace instead.
  return Bunch(x=xsc, y=ysc)
/home/reflex/.conda/envs/fastai/lib/python3.6/site-packages/plotnine/facets/layout.py:147: MatplotlibDeprecationWarning: The Bunch class was deprecated in Matplotlib 3.0 and will be removed in 3.2. Use types.SimpleNamespace instead.
  return Bunch(x=xsc, y=ysc)
/home/reflex/.conda/envs/fastai/lib/python3.6/site-packages/plotnine/facets/layout.py:147: MatplotlibDeprecationWarning: The Bunch class was deprecated in Matplotlib 3.0 and will be removed in 3.2. Use types.SimpleNamespace instead.
  return Bunch(x=xsc, y=ysc)
/home/reflex/.conda/envs/fastai/lib/python3.6/site-packages/plotnine/facets/layout.py:147: MatplotlibDeprecationWarning: The Bunch class was deprecated in Matplotlib 3.0 and will be removed in 3.2. Use types.Simple

/home/reflex/.conda/envs/fastai/lib/python3.6/site-packages/plotnine/facets/layout.py:147: MatplotlibDeprecationWarning: The Bunch class was deprecated in Matplotlib 3.0 and will be removed in 3.2. Use types.SimpleNamespace instead.
  return Bunch(x=xsc, y=ysc)
/home/reflex/.conda/envs/fastai/lib/python3.6/site-packages/plotnine/facets/layout.py:147: MatplotlibDeprecationWarning: The Bunch class was deprecated in Matplotlib 3.0 and will be removed in 3.2. Use types.SimpleNamespace instead.
  return Bunch(x=xsc, y=ysc)
/home/reflex/.conda/envs/fastai/lib/python3.6/site-packages/plotnine/facets/layout.py:147: MatplotlibDeprecationWarning: The Bunch class was deprecated in Matplotlib 3.0 and will be removed in 3.2. Use types.SimpleNamespace instead.
  return Bunch(x=xsc, y=ysc)
/home/reflex/.conda/envs/fastai/lib/python3.6/site-packages/plotnine/facets/layout.py:147: MatplotlibDeprecationWarning: The Bunch class was deprecated in Matplotlib 3.0 and will be removed in 3.2. Use types.Simple

In [ ]:
data = json.load(open(filename))